In [1]:
from coco_text import COCO_Text
import coco_utils
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
ct = COCO_Text('COCO_Text/COCO_Text.json')

loading annotations into memory...
0:00:04.091645
creating index...
index created!


In [31]:
def gen_images(ct, coco_id):
    fname = ct.imgs[coco_id]['file_name']
    image = cv2.imread(
        os.path.join('COCO_Text/images/', fname),
    )
    annotation = np.zeros(image.shape[:-1], np.uint8)
    weight = np.ones(image.shape[:-1], np.uint8) * 255

    for ann in ct.imgToAnns[coco_id]:
        poly = np.array(ct.anns[ann]['polygon'], np.int32).reshape((4,2))

        if ct.anns[ann]['legibility'] == 'legible':
            # draw only legible bbox/polygon
            cv2.fillConvexPoly(annotation, poly, 255)
        else:
            # 0 weight if it is illegible
            cv2.fillConvexPoly(weight, poly, 0)

    return [image, annotation, weight]

valid_anns = [ann for ann in ct.anns]
imgs = [
    i for i in ct.imgs if any(
        ann for ann in ct.imgToAnns[i]
            if ct.anns[ann]['legibility'] == 'legible'
    )
]

In [32]:
coco_id = np.random.choice(imgs)

In [33]:
image, annotation, weight = gen_images(ct, coco_id)
image_rgb = image[:,:,::-1]
annotation_rgb = cv2.cvtColor(annotation, cv2.COLOR_GRAY2BGR)[:,:,::-1]
weight_rgb = cv2.cvtColor(weight, cv2.COLOR_GRAY2BGR)[:,:,::-1]
plt.imshow(np.hstack((image_rgb, annotation_rgb, weight_rgb)))

In [34]:
ann = np.random.choice(ct.imgToAnns[coco_id])
window = coco_utils.get_window(image.shape[:-1], ct.anns[ann])
print window['slice']
print window['pad']
image, annotation, weight = coco_utils.crop_resize([image, annotation, weight], window, 256)

[slice(109, 297, None), slice(137, 325, None)]
((0, 0), (0, 0))


In [35]:
image_rgb = image[:,:,::-1]
annotation_rgb = cv2.cvtColor(annotation, cv2.COLOR_GRAY2BGR)[:,:,::-1]
weight_rgb = cv2.cvtColor(weight, cv2.COLOR_GRAY2BGR)[:,:,::-1]
plt.imshow(np.hstack((image_rgb, annotation_rgb, weight_rgb)))